In [ ]:
import pandas as pd
import numpy as np

import nltk 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
from nltk.tokenize import word_tokenize

In [ ]:
# GOAL: Using supervised classification, prune out all of the newsletter-esque posts that are irrelevant to my goals.

In [ ]:
LOADING = True

In [ ]:
if LOADING:
    messages = pd.read_parquet("data/all_messages.parquet")
else:
    WORKING_DIR = '/kaggle/input/'
    ticker_dir = WORKING_DIR + 'short-financial-messages/data/'
    PATH = ticker_dir
    EXT = "*.csv"

    all_csv_files = [file
                     for path, subdir, files in os.walk(PATH)
                     for file in glob(os.path.join(path, EXT))]

    parse_csv = lambda file: pd.read_csv(file, parse_dates=['created_at'])

    messages = pd.concat((parse_csv(f) for f in all_csv_files), ignore_index=True, sort=False)

    messages.set_index('id', inplace=True)
    messages.index = messages.index.map(str)
    messages = messages[~messages.index.duplicated(keep='first')]

    filter_urls = lambda text: re.sub(r"http\S+", "", str(text))
    messages['body'] = messages['body'].apply(filter_urls)

    messages["sentiment"] = messages["sentiment"].replace({-1: 0})
    messages.to_parquet("all_messages")

In [ ]:
messages

In [ ]:
labeled = messages[messages['sentiment'] != -69]
labeled["is_spam"] = -69 * np.ones(len(labeled), dtype=np.int)

In [ ]:
lemmatiser = WordNetLemmatizer()
stop_words = stopwords.words('english')

def preprocess_text(text):
    stop_words = stopwords.words('english')

    no_punc = "".join([char for char in text if char not in string.punctuation])
    words = nltk.word_tokenize(no_punc)
    filtered_words = [word for word in words if word not in stop_words]

    stemmed = [lemmatiser.lemmatize(word) for word in filtered_words]
    return stemmed

In [ ]:
mask = (labeled['body'].str.contains("smartoptions® unusual activity alert"))
d = labeled[mask]
labeled.loc[d.index, "is_spam"] = 1
labeled.loc[d.index]